In [1]:
import numpy as np
import cv2 as cv
import glob
import winsound, time, os, sys


In [2]:
# find the data file with extension .npz
outfile = glob.glob("*.npz")
outfile = outfile[0]

# load the npz file with objpoints and imgpoints array
npzfile = np.load(outfile)

# extract the two arrays...
objpoints = npzfile["objpoints"]
imgpoints = npzfile["imgpoints"]

len(objpoints)
len(imgpoints)


627

In [9]:
# use this cell to modify how many images are used as reference...
ref_images = 20    # how many images to use? 

objp = objpoints[:ref_images+1]
imgp = imgpoints[:ref_images+1]

# camera calibration

startTime = time.time()

frame_width  = w = 1920
frame_height = h = 1080

# determine camera matrix, distortion coefficients, 
# rotation and translation vectors
rms, camera_matrix, dist_coefs, rvecs, tvecs = cv.calibrateCamera(
                                                objp, imgp, 
                                                (frame_width,frame_height), None, None)

print("\nRMS", rms, "\n")
print("Camera Matrix: \n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel(), "\n")

# compute focal lengths
fx = camera_matrix[0][0]   # focal length in x-direction
fy = camera_matrix[1][1]   # focal length in y-direction
W  = 4                 # sensor width in mm    6.3 x 4.7mm
H  = 3                # sensor height in mm

Fx = fx * W/w
Fy = fy * H/h

print("Focal length in x = {:.2f} mm".format(Fx))
print("Focal length in y = {:.2f} mm".format(Fy))
winsound.Beep(440, 1000)
print("Processed {:.0f} images in {:.1f} s\n".format(ref_images, time.time()-startTime))


RMS 0.2783310003014989 

Camera Matrix: 
 [[1.91457020e+03 0.00000000e+00 9.83044334e+02]
 [0.00000000e+00 1.93890823e+03 5.84097293e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coefficients:  [ 2.21572445e-01 -1.81028987e+00  1.65591677e-03 -2.19022404e-03
  9.31437101e+00] 

Focal length in x = 3.99 mm
Focal length in y = 5.39 mm
Processed 20 images in 2.8 s

983.0443338212475


In [11]:
# refine the camera matrix
newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix, 
                                                dist_coefs, (w,h), 1, (w,h))

print("New Camera Matrix: \n", newcameramtx)
print("ROI = ", roi)

New Camera Matrix: 
 [[2.24438110e+03 0.00000000e+00 9.55885886e+02]
 [0.00000000e+00 2.02282898e+03 5.19772218e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
ROI =  (201, 136, 1512, 779)


In [8]:
def cameraMatrix(no_images, npzFileName, w = 1920, h = 1080):
    # timing of the function
    startTime = time.time() 
    
    # Find and read the .npz file with data in
    npzfile = np.load(npzFileName)
    
    # extract the object and image points
    objpoints = npzfile["objpoints"]
    imgpoints = npzfile["imgpoints"]
    objp      = objpoints[:no_images+1]   # only take the first n no. images
    imgp      = imgpoints[:no_images+1]
    
    # determine the RMS projection error, camera matrix, 
    # distortion coefficients, rotation and translation vectors
    rms, camera_matrix, dist_coefficients, rvecs, tvecs = cv.calibrateCamera(
                                                          objp, imgp, 
                                                          (w,h), None, None)
    
    #fovx, fovy, focalLength, principalPoint, aspectRatio = cv.calibrationMatrixValues(
    #                                                       camera_matrix, (w,h), 
    #                                                      sensorWidth, sensorHeight
    
    
    print("Processed {:.0f} images in {:.1f} s".format(no_images, time.time()-startTime))
    print("\nRMS", rms, "\n")
    print("Camera Matrix: \n", camera_matrix)
    print("distortion coefficients: ", dist_coefficients.ravel(), "\n")
    
    # refine the camera matrix
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix, 
                                                dist_coefficients, (w,h), 1, (w,h))
    
    print("Optimised Camera Matrix: \n", newcameramtx, "\n \n")
    print("################################################################ \n")
    
    return no_images, rms, camera_matrix, dist_coefficients, newcameramtx
    



In [9]:
imgs, RMS, cameramtx, dist_coefficients, optimalCameraMtx = [], [], [], [], []

for no_images in np.arange(10, 60, 10):
    images, rms, camera_matrix, dist_coefs, newcameramtx = cameraMatrix(no_images, outfile)
    imgs.append(images)
    RMS.append(rms)
    cameramtx.append(camera_matrix)
    dist_coefficients.append(dist_coefs)
    optimalCameraMtx.append(newcameramtx)

Processed 10 images in 0.4 s

RMS 0.2794895255888173 

Camera Matrix: 
 [[1.91789270e+03 0.00000000e+00 9.69257760e+02]
 [0.00000000e+00 1.93170636e+03 5.55808633e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coefficients:  [ 2.28628219e-01 -2.07902501e+00 -1.05829844e-04 -3.46736889e-03
  1.04211805e+01] 

Optimised Camera Matrix: 
 [[2.03841101e+03 0.00000000e+00 9.63430829e+02]
 [0.00000000e+00 1.95113928e+03 5.55540827e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 
 

################################################################ 

Processed 20 images in 1.8 s

RMS 0.2783310003014989 

Camera Matrix: 
 [[1.91457020e+03 0.00000000e+00 9.83044334e+02]
 [0.00000000e+00 1.93890823e+03 5.84097293e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion coefficients:  [ 2.21572445e-01 -1.81028987e+00  1.65591677e-03 -2.19022404e-03
  9.31437101e+00] 

Optimised Camera Matrix: 
 [[2.03603186e+03 0.00000000e+00 9.74913702e+02]
 [0.00000000e+00 1.9593188

In [17]:
a = np.arange(10,60,10)
b = a.tolist()

0

SyntaxError: non-default argument follows default argument (<ipython-input-18-b31db5d6dd45>, line 1)